In [ ]:
import MesoPy
import pandas as pd
import numpy as np
import missingno as msno
import os
import matplotlib.pyplot as plt
import station_pipeline
import chart_studio.plotly as py

In [ ]:
df = pd.read_csv('ipw_inputs/irwin_model.v1',
                delimiter=' ',
                header=None)
df.info()

In [ ]:
df.head()

In [ ]:
df.rename(columns={0: 'elapsed time since start of model run (hours)',
                1: 'net allwave rad (W/m^2)',
                2: 'sensible heat transfer (W/m^2)',
                3: 'latent heat exchange (W/m^2)',
                4: 'snow/soil heat exchange (W/m^2)',
                5: 'advected heat from precip. (W/m^2)',
                6: 'sum of e.b. terms for snowcover (W/m^2)',
                7: 'heat exchange between snow layers (W/m^2)',
                8: 'sum of e.b. terms for surface layer (W/m^2)',
                9: 'surface layer cold content (J/m^2)',
                10: 'lower layer cold content (J/m^2)',
                11: 'snowcover cold content (J/m^2)',
                12: 'evaporation (kg/m^2)',
                13: 'melt (kg/m^2)',
                14: 'predicted runoff (kg, or mm/m^2)',
# check if "runoff error" is being output. 
                15: 'predicted depth of surface layer (m)',
                16: 'predicted   "   of lower layer (m)',
                17: 'predicted   "   of snowcover (m)',
                18: 'predicted average snow density (kg/m^3)',
                19: 'predicted specific mass of surface layer (kg/m^2)',
                20: 'predicted    "      "   of lower layer (kg/m^2)',
                21: 'predicted    "      "   of snowcover (kg/m^2)',
                22: 'predicted liquid H2O in snowcover (kg/m^2)',
                23: 'predicted temperature of surface layer (C)',
                24: 'predicted      "      of lower layer (C)',
                25: 'predicted average temp of snowcover (C)'}, inplace=True)

In [ ]:
df.columns()

In [ ]:
df.plot(subplots=True, figsize=(30,40))

In [ ]:
fig = station_pipeline.html_chart(df, height=4000)
fig.show()
fig.write_html('plots/irwin.html')

## SNOTEL comparison

In [ ]:
df2 = pd.read_csv('raw_data/Irwin_WY20.csv',
                skiprows=[0,1,2,3,4,5,7],
                 na_values=['NAN','NA','N/A','NaN']
                 )

In [ ]:
dfir = df2.iloc[:,0:10]
dfir.index = pd.to_datetime(dfir['Date_Time']); dfir.info()

In [ ]:
# all raw data in MST, so dropping tz for now
dfir = dfir.tz_localize(None)
dfir = dfir.dropna(how='all')

#drop_duplicates is not used here. it can result in unexpected behavior in timeseries df.
dfir = dfir[~dfir.index.duplicated()]
# recast index
dfir = dfir.asfreq('1H')
dfir.info()

In [ ]:
dfir = dfir[:'20200630 22:00'].copy()

In [ ]:
df.info()

In [ ]:
df['predicted   "   of snowcover (m)'].plot()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.index = pd.date_range(start='2019-10-01 01:00:00', end='2020-06-30 23:00:00', freq='1H')

In [ ]:
dfir['modeled_snow_depth'] = df['predicted   "   of snowcover (m)'].copy()

In [ ]:
dfir['modeled_snow_depth']

In [ ]:
dfir['snow_depth_m'] = dfir['snow_depth_set_1'] / 1000

In [ ]:
ax = dfir[['snow_depth_m','modeled_snow_depth']].plot(figsize=(20,10))

fig = ax.get_figure()

plt.savefig('plots/irwin_depth_compare.png', dpi=150, edgecolor='white', facecolor='white')

In [ ]:
df.info()

In [ ]:
#df = df.round(3)
#df.to_csv('output/Irwin_wy2020_snobal_out_20201118.csv')